In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4o-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [88]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4o-mini")

In [68]:
def compute_agent_cost(agent):
    input_token_price = 0.15 / 1000000
    output_token_price = 0.6 / 1000000
    return (
        input_token_price * agent.monitor.get_total_token_counts()["input"]
        + output_token_price * agent.monitor.get_total_token_counts()["output"]
    )

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [9]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [10]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can help with a wide range of tasks! I can provide answers to questions, perform calculations, analyze data, solve problems, generate content, and even transform images if needed. Whether you need an explanation, a numerical result, or step-by-step help on a task, I’m here to assist you.

Cost of the agent:  0.00021854999999999997


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [11]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I'm sorry, but I can’t browse the internet or access real-time content from external websites like https://www.swissinfo.ch/eng/ as my current functionality is limited to retrieving and processing information up to my last training cutoff. For the most recent news, please visit the website directly or use a current news source.

Cost of the agent:  0.00049155


In [12]:
agent.visualize()

ToolCallingAgent | o3-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [13]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [14]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | o3-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [15]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

Here are some of the most recent headlines featured on the homepage of SWI swissinfo:

• “Fatal Swiss ski touring accidents reach five‐year peak” – Published on Mar 27, 2025 at 13:10
• “Tariq Ramadan rape trial in Paris ‘set for March’” – Published on Mar 27, 2025 at 11:41
• “UN ‘deeply shocked’ by army strikes in North Darfur” – Published on Mar 27, 2025 at 10:54
• “UBS struggles with technical problems in payment transactions” – Published on Mar 27, 2025 at 10:01
• “Swiss railway line to Zermatt reopens after rockfall” – Published on Mar 27, 2025 at 09:53
• “Virtual Swiss agents cleared to infiltrate chatrooms” – Published on Mar 27, 2025 at 09:37

These headlines are part of the “Latest news” section on the site. (Note: The page includes many other stories and topics, but the above items are among the most current.)

Cost of the agent:  0.00303285


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [16]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

In [17]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [18]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [19]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
    verbosity_level=0,
)
agent.visualize()

ToolCallingAgent | o3-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [20]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

7

Cost of the agent:  0.00038415


In [21]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [22]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [23]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | o3-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [24]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - o3-mini ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 1.54 seconds| Input tokens: 999 | Output tokens: 33]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.28 seconds| Input tokens: 2,092 | Output tokens: 56]

7

Cost of the agent:  0.0003474


## Code Agent

In [89]:
from smolagents import DuckDuckGoSearchTool

web_search = DuckDuckGoSearchTool()
agent = ToolCallingAgent(
    model=model,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [90]:
output = agent.run(
    "You are an agent that can study financial market. Today's date is 26th March 2025. "+
    "What is the gain (in absolute value) that Nvidia stock made in the last year?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 26th March 2025.What is the gain (in absolute │
│ value) that Nvidia stock made in the last year?                                                                 │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'Nvidia stock price on March 26 2024'}                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|NVIDIA Corporation (NVDA) Stock Historical Prices & Data - Yahoo 
Finance](https://finance.yahoo.com/quote/NVDA/history/)
Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when 
NVIDIA Corporation stock was issued. News ... Dec 26, 2024: 139.70: 140.85: 137.73:

|NVIDIA Stock Price History - Investing.com](https://www.investing.com/equities/nvidia-corp-historical-data)
Access NVIDIA stock price history with daily data, historical prices, all-time highs, and stock chart history. 
Download and analyze trends easily. Investing.com - Financial Markets Worldwide

|NVIDIA - 26 Year Stock Price History | NVDA | 
MacroTrends](https://www.macrotrends.net/stocks/charts/NVDA/nvidia/stock-price-history)
Historical daily share price chart and data for NVIDIA since 1999 adjusted for splits and dividends. The latest 
closing stock price for NVIDIA as of March 24, 2025 is 121.41.. The all-time high NVIDIA stock closing price was 
149.43 on January 06, 2025.; The NVIDIA 52-week high stock price is 195.95, which is 61.4% above the current share 
price.; The NVIDIA 52-week low stock price is 75.61 ...

|Nvidia Stock Price In March 2024 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-in-march-2024)
The closing price for NVIDIA (NVDA) in March 2024 was $90.33, on March 28, 2024. It was up 13% for the month. The 
latest price is $121.67.

|NVIDIA (NVDA) Historical Stock Price Data - Stock Analysis](https://stockanalysis.com/stocks/nvda/history/)
A complete stock price history for NVIDIA Corporation (NVDA) stock, starting from its first trading day. Includes 
open, high, low, close and volume.

|NVIDIA Corporation - Stock Info - Historical Price 
Lookup](https://investor.nvidia.com/stock-info/historical-price-lookup/default.aspx)
We intend to use our @NVIDIA Twitter account, NVIDIA Facebook page, NVIDIA LinkedIn page and company blog as a 
means of disclosing information about our company, our services and other matters and for complying with our 
disclosure obligations under Regulation FD. The information we post through these social media channels may be 
deemed material.

|NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Nvidia stock price prediction for tomorrow, near days, this week and this month. ... previous day at $120.69, which
is $9.59 lower than 30 days ago and $5.26 higher than 7 days ago. Nvidia Stock Price Forecast. Nvidia Stock Price 
By Day. Date: ... on Friday, March, 28: 123.75 dollars, maximum 133.65, minimum 113.85. Nvidia stock price forecast
...

|NVIDIA (NVDA) Stock Chart and Price History 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/chart/)
View NVIDIA (NASDAQ:NVDA) historical prices, past price performance, and an advanced NVDA stock chart at 
MarketBeat. ... NVIDIA Stock Price Performance. 5 Day Performance +1.82%. 1 Month Performance-4.69%. 3 Month 
Performance-13.93%. ... NVDA Stock Chart for Wednesday, March, 26, 2025. NVDA Chart by TradingView. Remove Ads.

|Why Nvidia (NVDA) Shares Are Tumbling Today - Yahoo 
Finance](https://finance.yahoo.com/news/why-nvidia-nvda-shares-tumbling-160534659.html)
Unlock stock picks and a broker-level newsfeed that powers Wall Street. ... Wed, Mar 26, 2025, 12:05 PM 1 min read.
In This Article: NVDA . March 26 - Nvidia ... Analysts estimate that Nvidia ...

|Nvidia Stock Just Did This for the First Time in 3 Years. Here's What 
...](https://www.fool.com/investing/2025/03/25/nvidia-just-did-this-for-first-time-in-3-years/)
Price as of March 26, 2025, 3:58 p.m. ET Nvidia ( NVDA -5.59% ) amazed investors over the past few years, climbing 
with what seemed like nonstop momentum to reach record levels.

[Step 1: Duration 2.16 seconds| Input tokens: 1,134 | Output tokens: 25]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '204.11'}                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 204.11

[Step 5: Duration 1.79 seconds| Input tokens: 15,798 | Output tokens: 215]

204.11

Cost of the agent:  0.0024986999999999995


In [91]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [92]:
output = agent.run(
    "You are an agent that can study financial market. Today's date is 26th March 2025. "+
    "What is the gain (in absolute value) that Nvidia stock made in the last year?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 26th March 2025.What is the gain (in absolute │
│ value) that Nvidia stock made in the last year?                                                                 │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_price_2024 = web_search(query="Nvidia stock price March 26 2024")                                         
  print("Nvidia Stock Price on March 26, 2024:", nvidia_price_2024)                                                
                                                                                                                   
  nvidia_price_2025 = web_search(query="Nvidia stock price March 26 2025")                                         
  print("Nvidia Stock Price on March 26, 2025:", nvidia_price_2025)                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Nvidia Stock Price on March 26, 2024: ## Search Results

[NVIDIA Corporation (NVDA) Stock Historical Prices & Data - Yahoo 
Finance](https://finance.yahoo.com/quote/NVDA/history/)
Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when 
NVIDIA Corporation stock was issued. News ... Dec 26, 2024: 139.70: 140.85: 137.73:

[NVIDIA Stock Price History - Investing.com](https://www.investing.com/equities/nvidia-corp-historical-data)
Access NVIDIA stock price history with daily data, historical prices, all-time highs, and stock chart history. 
Download and analyze trends easily. Investing.com - Financial Markets Worldwide

[NVIDIA - 26 Year Stock Price History | NVDA | 
MacroTrends](https://www.macrotrends.net/stocks/charts/NVDA/nvidia/stock-price-history)
Historical daily share price chart and data for NVIDIA since 1999 adjusted for splits and dividends. The latest 
closing stock price for NVIDIA as of March 24, 2025 is 121.41.. The all-time high NVIDIA stock closing price was 
149.43 on January 06, 2025.; The NVIDIA 52-week high stock price is 195.95, which is 61.4% above the current share 
price.; The NVIDIA 52-week low stock price is 75.61 ...

[Nvidia Stock Price In March 2024 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-in-march-2024)
The closing price for NVIDIA (NVDA) in March 2024 was $90.33, on March 28, 2024. It was up 13% for the month. The 
latest price is $121.67.

[NVIDIA (NVDA) Historical Stock Price Data - Stock Analysis](https://stockanalysis.com/stocks/nvda/history/)
A complete stock price history for NVIDIA Corporation (NVDA) stock, starting from its first trading day. Includes 
open, high, low, close and volume.

[NVIDIA Corporation - Stock Info - Historical Price 
Lookup](https://investor.nvidia.com/stock-info/historical-price-lookup/default.aspx)
We intend to use our @NVIDIA Twitter account, NVIDIA Facebook page, NVIDIA LinkedIn page and company blog as a 
means of disclosing information about our company, our services and other matters and for complying with our 
disclosure obligations under Regulation FD. The information we post through these social media channels may be 
deemed material.

[NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Nvidia stock price prediction for tomorrow, near days, this week and this month. ... previous day at $120.69, which
is $9.59 lower than 30 days ago and $5.26 higher than 7 days ago. Nvidia Stock Price Forecast. Nvidia Stock Price 
By Day. Date: ... on Friday, March, 28: 123.75 dollars, maximum 133.65, minimum 113.85. Nvidia stock price forecast
...

[NVIDIA (NVDA) Stock Chart and Price History 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/chart/)
View NVIDIA (NASDAQ:NVDA) historical prices, past price performance, and an advanced NVDA stock chart at 
MarketBeat. ... NVIDIA Stock Price Performance. 5 Day Performance +1.82%. 1 Month Performance-4.69%. 3 Month 
Performance-13.93%. ... NVDA Stock Chart for Wednesday, March, 26, 2025. NVDA Chart by TradingView. Remove Ads.

[NVIDIA (NVDA) Stock Forecast & Price Targets - Stock Analysis](https://stockanalysis.com/stocks/nvda/forecast/)
Stock forecasts and price targets for NVIDIA Corporation (NVDA) stock, including current and historical analyst 
ratings and detailed revenue and earnings estimates. ... FY 2024 FY 2025 FY 2026 FY 2027 FY 2028 FY 2029 FY 2030 ; 
Period Ending Jan 31, 2021 Jan 30, 2022 Jan 29, 2023 Jan 28, 2024 Jan 26, 2025 Jan 26, 2026 Jan 26, 2027 Jan 26, 
2028 ...

[NVIDIA (NVDA) Stock Price & Overview - Stock Analysis](https://stockanalysis.com/stocks/nvda/)
Get a real-time NVIDIA Corporation (NVDA) stock price with an overview of financials, statistics, forecasts, charts
and more. ... Feb 26, 2025: About NVDA. ... In 2024, NVIDIA's revenue was $130.50 billion, an increase of 114.20% 
compared to the previous year's $60.92 billion. ...
Nvidia Sto

[Step 1: Duration 5.96 seconds| Input tokens: 2,129 | Output tokens: 189]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  price_2024 = 90.33                                                                                               
  price_2025 = 121.67                                                                                              
  absolute_gain = abs(price_2025 - price_2024)                                                                     
  final_answer(absolute_gain)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 31.340000000000003

[Step 2: Duration 2.96 seconds| Input tokens: 6,575 | Output tokens: 313]

Cost of the agent:  0.0011740499999999998


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [93]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to run web search to obtain information. Call it as a 
        │   function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
            │              │ your query (think a Google search) then   │ perform.                                 │
            │              │ returns the top search results.           │                                          │
            │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
            │              │ problem.                                  │ problem                                  │
            └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘

In [94]:
output = manager_agent.run(
    "You are an agent that can study financial market. Today's date is 26th March 2025. "+
    "What is the gain (in absolute value) that Nvidia stock made in the last year?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(manager_agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 26th March 2025. What is the gain (in         │
│ absolute value) that Nvidia stock made in the last year?                                                        │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for the current stock price of Nvidia                                                                   
  current_price = search("Nvidia stock price March 26 2025")                                                       
  print("Current Nvidia stock price:", current_price)                                                              
                                                                                                                   
  # Search for the stock price of Nvidia from one year ago                                                         
  previous_price = search("Nvidia stock price March 26 2024")                                                      
  print("Nvidia stock price March 26 2024:", previous_price)                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'current_price = search("Nvidia stock price March 26 2025")' due to: 
InterpreterError: It is not permitted to evaluate other functions than the provided tools or functions 
defined/imported in previous code (tried to execute search).

[Step 1: Duration 3.36 seconds| Input tokens: 2,178 | Output tokens: 176]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Stock prices                                                                                                   
  current_price = 120.69                                                                                           
  previous_price = 90.33                                                                                           
                                                                                                                   
  # Calculate the absolute gain                                                                                    
  absolute_gain = current_price - previous_price                                                                   
  print("The absolute gain in Nvidia's stock over the last year is:", absolute_gain)                               
  final_answer(absolute_gain)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The absolute gain in Nvidia's stock over the last year is: 30.36

Out - Final answer: 30.36

[Step 3: Duration 2.80 seconds| Input tokens: 10,611 | Output tokens: 635]

Cost of the agent:  0.00197265


In [ ]:
agent.write_memory_to_messages()

## Visualize Your Agent

In [ ]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="Manager Agent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [ ]:
import requests

class PDFExtractorAPI:
    def __init__(self):
        pass

    def extract_text_and_images(self, pdf_file_path: str) -> tuple[str, list]:
        BASE_URL = "https://compute.datascience.ch/vlm2vec-innovation"
        # Send the request with the file directly
        with open(pdf_file_path, "rb") as pdf_file:
            files = {"pdf_file": (pdf_file_path, pdf_file, "application/pdf")}
            response = requests.post(
                f"{BASE_URL}/v1/pdf_to_markdown",
                files=files
            )

        # Process the response
        result = response.json()
        markdown_text = result["markdown"]
        images = result["images"]

        return result, markdown_text, images


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractorAPI().extract_text_and_images(pdf_file_path)
    return text

In [ ]:
tmp = read_pdf_tool("../data/hypoxy_stat_1page.pdf")
tmp

In [ ]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

In [ ]:
output = agent.run(
    "Can you read the PDF file at '../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [ ]:
from src.data_classes import Chunk
from src.data_processing import SimpleChunker, PDFExtractorAPI
from src.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from src.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [ ]:
data_folder = "../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [ ]:
data_extractor = PDFExtractorAPI()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [ ]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [ ]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [ ]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_documents(chunks, embeddings)

In [ ]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

### Toolify

In [ ]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    ## Fill the docstring here
    
    return retriever.retrieve(query, number_of_chunks)

# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

In [ ]:
# Call the agent
output = rag_agent.run("According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?")

In [ ]:
rag_agent.write_memory_to_messages()[0]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents
